In [111]:
step = Mysql()

### 사용자 이메일과 사용자 이름 ID 매핑

In [78]:
sql = """select inviter_id ,email, profile_nickname from tbl_friend;"""
df = step.SQL(sql)

In [80]:
idx2name = {k[0]: k[2] for k in df.iloc}
name2idx = {value: key for key,value in idx2name.items()}
mail2name = {k[1]: k[2] for k in df.iloc}
name2mail = {value: key for key,value in mail2name.items()}

### 1.
### 보낸 상대평점 평균: 특정 기간 동안( 특정인에게 보낸 나의 평점 평균 / 내가 보낸 모든 평점 평균)

In [94]:
sql = f'''select F1.profile_nickname AS host_name, F2.profile_nickname AS guest_name, AVG(FE.vote) as avg_vote from tbl_feedback_history FE
JOIN tbl_friend F1 ON FE.host_id = F1.inviter_id
JOIN tbl_friend F2 ON FE.guest_id = F2.inviter_id
WHERE F1.profile_nickname = "허영호" AND F2.profile_nickname = "정유철 교수 금오공대";
'''
print(step.SQL(sql))

#내가보낸 모든 평점 평균
sql = f'''SELECT i.name, AVG(f.vote) as avg_vote
FROM tbl_inviter i
INNER JOIN tbl_feedback_history f ON i.inviter_id = f.host_id COLLATE utf8mb4_unicode_ci
WHERE i.name = "허영호"  
GROUP BY i.name;'''

print(step.SQL(sql))

     0            1       2
0  허영호  정유철 교수 금오공대  4.2892
     0       1
0  허영호  4.2201


### 2.
### 받은 상대평점 평균: 특정 기간 동안 (특정인으로 부터 받은 나의 평점 평균 / 특정인이 보낸 모든 평점 평균)

In [99]:
sql = f'''select F1.profile_nickname AS host_name, F2.profile_nickname AS guest_name, AVG(FE.vote) as avg_vote from tbl_feedback_history FE
JOIN tbl_friend F1 ON FE.host_id = F1.inviter_id
JOIN tbl_friend F2 ON FE.guest_id = F2.inviter_id
WHERE F1.profile_nickname = "정유철 교수 금오공대" AND F2.profile_nickname = "허영호";
'''
print(step.SQL(sql))

#내가보낸 모든 평점 평균
sql = f'''SELECT F.profile_nickname, AVG(FE.vote) as avg_vote
FROM tbl_friend F
INNER JOIN tbl_feedback_history FE ON F.inviter_id = FE.host_id COLLATE utf8mb4_unicode_ci
WHERE F.profile_nickname = "정유철 교수 금오공대" ; 
'''

print(step.SQL(sql))

             0    1       2
0  정유철 교수 금오공대  허영호  3.0964
             0       1
0  정유철 교수 금오공대  3.9890


### 3. 
### 보낸 업무상대평점 평균: (특정업무에 대해 보낸 나의 평점 평균/ 내가 보낸 모든 평점 평균)

In [109]:
#특정 업무에 대해 보낸 나의 평점 평균
sql = '''
select F.profile_nickname , S.schedule_id , AVG(FE.vote) as avg_vote from tbl_schedule_todo S
join tbl_feedback_history FE on S.schedule_id = FE.schedule_id COLLATE utf8mb4_unicode_ci
join tbl_friend F on F.inviter_id = FE.host_id COLLATE utf8mb4_unicode_ci
where F.profile_nickname ="허영호" and S.schedule_id = 742717624;
'''
a = step.SQL(sql)

#내가 보낸 모든 평점 평균
sql = f'''SELECT F.profile_nickname, AVG(FE.vote) as avg_vote
FROM tbl_friend F
INNER JOIN tbl_feedback_history FE ON F.inviter_id = FE.host_id COLLATE utf8mb4_unicode_ci
WHERE F.profile_nickname = "허영호" ; 
'''
b = step.SQL(sql)

print(f'{a} \n {b}')

(('허영호', '742717624', Decimal('5.0000')),) 
 (('허영호', Decimal('4.7126')),)


### 4.
### 보낸 일정완료율: (내가 완료한 일정 수/ 내가 생성한 일정 수)

In [112]:
sql = '''
select DISTINCT F.profile_nickname ,S.schedule_id   from tbl_schedule_todo S
join tbl_friend F on F.inviter_id = S.inviter_id COLLATE utf8mb4_unicode_ci
where F.profile_nickname = "허영호";
'''
a = len(step.SQL(sql))

#완료 판단 기준이 없어서 complete_time이 NULL이 아닐 경우
sql = '''  
select DISTINCT  F.profile_nickname ,S.schedule_id   from tbl_schedule_todo S
join tbl_friend F on F.inviter_id = S.inviter_id COLLATE utf8mb4_unicode_ci
where F.profile_nickname = "허영호" and S.complete_time is not NULL ;
'''
b = len(step.SQL(sql))

print(b/a)

0.327319587628866


### 5.
### 보낸 일정참여율: (일정 별 일정에 참여한 참여자 총 수의 합 / 일정 별 초대한 참여자 총 수의 합)

In [145]:
sql = '''
select S.title,  S.invitees  from tbl_schedule_todo S
join tbl_friend F on F.inviter_id = S.inviter_id 
where F.profile_nickname = "허영호" and S.amount_of_invitees != 0;
'''
a = step.SQL(sql)

sql = '''
select SUM(S.amount_of_invitees)  from tbl_schedule_todo S
join tbl_friend F on F.inviter_id = S.inviter_id 
where F.profile_nickname = "허영호" and S.amount_of_invitees != 0;
'''
b = len(step.SQL(sql))


In [148]:
import json
count, total = 0, 0
for line in a.iloc:
    json_list = json.loads(line[1])
    if len(json_list) > 0:
        for k in json_list:
            total += 1
            count += k["join"]
    
print(count / total)

0.9087947882736156


### 6.
### 보낸 피드백율: 특정 기간동안 (내가 보낸 피드백 수/ 내가 요청받은 피드백 수)

In [160]:
sql = '''
select count(*) from tbl_feedback_history FE 
join tbl_friend F on F.inviter_id = FE.guest_id 
where F.profile_nickname = "허영호" and vote!= 0;
'''
a = step.SQL(sql)

sql = '''
select count(*) from tbl_feedback_history FE 
join tbl_friend F on F.inviter_id = FE.guest_id 
where F.profile_nickname = "허영호";
'''
b = step.SQL(sql)

print(a.iloc[0][0]/ b.iloc[0][0])

0.908701610140459


### 7.
### 받은 일정참여율: 특정 기간동안 (내가 참여한 일정 수/ 내가 초대받은 일정 수)
### 

In [169]:
sql = '''
select S.title,  S.invitees  from tbl_schedule_todo S
join tbl_friend F on F.inviter_id = S.inviter_id 
where S.invitees is not NULL;
'''
a = step.SQL(sql)

In [170]:
import json
count, total = 0, 0
for line in a.iloc:
    json_list = json.loads(line[1])
    if len(json_list) > 0:
        for k in json_list:
            if k["name"] == "허영호":
                total += 1
                if k["join"] == 1:
                    count += 1
            
    
print(count / total)

0.4013452914798206


### 8.
### 받은 피드백율: 특정 기간 동안 (내가 요청 받은 피드백 수/ 내가 참여한 일정 수)
####  현재 과정으로는 1